Where is the mass affected?
    plot_calcs: used for the overall stellar mass function
        -will return bin_centers -> make these the specific ones to go with each bin that actually has something
    hist_calcs: definitely affected; but where to change them?
        being fed mass_dat... mass_dat is going into plot_calcs
            dictionary values and error are zero
Should be okay:
    bin_func: should be okay. median distance to nearest neighbor shouldn't be calculated if the bin is empty...  
    
    
Notes:
    Need a definitive bin_centers variable for plotting the medians, among other things.

In [35]:
from __future__ import division, absolute_import

import astropy.stats
import glob
import math
import matplotlib.pyplot as plt 
from matplotlib import ticker
from matplotlib.ticker import FormatStrFormatter
import numpy as np 
import os
import pandas as pd
from scipy import integrate,optimize,spatial

In [36]:
__author__     =['Victor Calderon']
__copyright__  =["Copyright 2016 Victor Calderon, Index function"]
__email__      =['victor.calderon@vanderbilt.edu']
__maintainer__ =['Victor Calderon']

def Index(directory, datatype):
    """
    Indexes the files in a directory `directory' with a
    specific data type.

    Parameters
    ----------
    directory: str
            Absolute path to the folder that is indexed.

    datatype: str
            Data type of the files to be indexed in the folder.

    Returns
    -------
    file_array: array_like 
            np.array of indexed files in the folder 'directory' 
            with specific datatype.

    Examples
    --------
    >>> Index('~/data', '.txt')
    >>> array(['A.txt', 'Z'.txt', ...])
    """
    assert(os.path.exists(directory))
    files = np.array(glob.glob('{0}/*{1}'.format(directory, datatype)))

    return files

In [37]:
def myceil(x, base=10):
    """
    Returns the upper-bound integer of 'x' in base 'base'.

    Parameters
    ----------
    x: float
        number to be approximated to closest number to 'base'

    base: float
        base used to calculate the closest 'largest' number

    Returns
    -------
    n_high: float
        Closest float number to 'x', i.e. upper-bound float.

    Example
    -------
    >>>> myceil(12,10)
      20
    >>>>
    >>>> myceil(12.05, 0.1)
     12.10000 
    """
    n_high = float(base*math.ceil(float(x)/base))

    return n_high

###############################################################################    

def myfloor(x, base=10):
    """
    Returns the lower-bound integer of 'x' in base 'base'

    Parameters
    ----------
    x: float
        number to be approximated to closest number of 'base'

    base: float
        base used to calculate the closest 'smallest' number

    Returns
    -------
    n_low: float
        Closest float number to 'x', i.e. lower-bound float.

    Example
    -------
    >>>> myfloor(12, 5)
    >>>> 10
    """
    n_low = float(base*math.floor(float(x)/base))

    return n_low

###############################################################################

def Bins_array_create(arr, base=10):
    """
    Generates array between [arr.min(), arr.max()] in steps of `base`.

    Parameters
    ----------
    arr: array_like, Shape (N,...), One-dimensional
        Array of numerical elements

    base: float, optional (default=10)
        Interval between bins

    Returns
    -------
    bins_arr: array_like
        Array of bin edges for given arr

    """
    base = float(base)
    arr  = np.array(arr)
    assert(arr.ndim==1)
    arr_min  = myfloor(arr.min(), base=base)
    arr_max  = myceil( arr.max(), base=base)
    bins_arr = np.arange(arr_min, arr_max+0.5*base, base)

    return bins_arr

In [38]:
def sph_to_cart(ra,dec,cz):
    """
    Converts spherical coordinates to Cartesian coordinates.

    Parameters
    ----------
    ra: array-like
        right-ascension of galaxies in degrees
    dec: array-like
        declination of galaxies in degrees
    cz: array-like
        velocity of galaxies in km/s

    Returns
    -------
    coords: array-like, shape = N by 3
        x, y, and z coordinates
    """
    cz_dist = cz/70. #converts velocity into distance
    x_arr   = cz_dist*np.cos(np.radians(ra))*np.cos(np.radians(dec))
    y_arr   = cz_dist*np.sin(np.radians(ra))*np.cos(np.radians(dec))
    z_arr   = cz_dist*np.sin(np.radians(dec))
    coords  = np.column_stack((x_arr,y_arr,z_arr))

    return coords

############################################################################

def calc_dens(n_val,r_val):
    """
    Returns densities of spheres with radius being the distance to the 
        nth nearest neighbor.

    Parameters
    ----------
    n_val = integer
        The 'N' from Nth nearest neighbor
    r_val = array-like
        An array with the distances to the Nth nearest neighbor for
        each galaxy

    Returns
    -------
    dens: array-like
        An array with the densities of the spheres created with radii
        to the Nth nearest neighbor.
    """
    dens = np.array([(3.*(n_val+1)/(4.*np.pi*r_val[hh]**3)) \
                     for hh in range(len(r_val))])

    return dens

In [39]:
def plot_calcs(mass,bins,dlogM):
    """
    Returns values for plotting the stellar mass function and 
        mass ratios

    Parameters
    ----------
    mass: array-like
        A 1D array with mass values, assumed to be in order
    bins: array=like
        A 1D array with the values which will be used as the bin edges
        by the histogram function
    dlogM: float-like
        The log difference between bin edges

    Returns
    -------
    bin_centers: array-like
        An array with the medians mass values of the mass bins
    mass-freq: array-like
        Contains the number density values of each mass bin
    ratio_dict: dictionary-like
        A dictionary with three keys, corresponding to the divisors
        2,4, and 10 (as the percentile cuts are based on these 
        divisions). Each key has the density-cut, mass ratios for
        that specific cut (50/50 for 2; 25/75 for 4; 10/90 for 10).
    """

    mass_counts, edges = np.histogram(mass,bins)
    bin_centers        = 0.5*(edges[:-1]+edges[1:])

    mass_freq  = mass_counts/float(len(mass))/dlogM

    ratio_dict = {}
    frac_val   = [2,4,10]

    yerr = []

    for ii in frac_val:
        ratio_dict[ii] = {}

        # Calculations for the lower density cut
        frac_data      = int(len(mass)/ii)
        frac_mass      = mass[0:frac_data]
        counts, edges  = np.histogram(frac_mass,bins)

        # Calculations for the higher density cut
        frac_mass_2       = mass[-frac_data:]
        counts_2, edges_2 = np.histogram(frac_mass_2,bins)

        # Ratio determination
        ratio_counts   = (1.*counts_2)/(1.*counts)
        ratio_dict[ii] = ratio_counts

        yerr.append((counts_2*1.)/(counts*1.)*\
            np.sqrt(1./counts + 1./counts_2))

    mass_freq_list     = [[] for xx in xrange(2)]
    mass_freq_list[0]  = mass_freq
    mass_freq_list[1]  = np.sqrt(mass_counts)/float(len(mass))/dlogM
    mass_freq          = np.array(mass_freq_list)

    ratio_dict_list    = [[] for xx in range(2)]
    ratio_dict_list[0] = ratio_dict
    ratio_dict_list[1] = yerr
    ratio_dict         = ratio_dict_list

    return bin_centers, mass_freq, ratio_dict

In [40]:
def bin_func(mass_dist,bins,kk,bootstrap=False):
    """
    Returns median distance to Nth nearest neighbor

    Parameters
    ----------
    mass_dist: array-like
        An array with mass values in at index 0 (when transformed) and distance 
        to the Nth nearest neighbor in the others
        Example: 6239 by 7
            Has mass values and distances to 6 Nth nearest neighbors  
    bins: array=like
        A 1D array with the values which will be used as the bin edges     
    kk: integer-like
        The index of mass_dist (transformed) where the appropriate distance 
        array may be found

    Optional
    --------
    bootstrap == True
        Calculates the bootstrap errors associated with each median distance 
        value. Creates an array housing arrays containing the actual distance 
        values associated with every galaxy in a specific bin. Bootstrap error
        is then performed using astropy, and upper and lower one sigma values 
        are found for each median value.  These are added to a list with the 
        median distances, and then converted to an array and returned in place 
        of just 'medians.'

    Returns
    -------
    medians: array-like
        An array with the median distance to the Nth nearest neighbor from 
        all the galaxies in each of the bins
    """
    frac_vals    = np.array([2,4,10])
    
    edges        = bins

    # print 'length bins:'
    # print len(bins)

    digitized    = np.digitize(mass_dist.T[0],edges)
    digitized   -= int(1)

    bin_nums          = np.unique(digitized)
    bin_nums_list     = list(bin_nums)

    if (len(bins)) in bin_nums_list:
        bin_nums_list.remove(len(bins))
    # print 'removed'
    # print bin_nums_list

    bin_nums = np.array(bin_nums_list)

    for ii in bin_nums:
        if len(mass_dist.T[kk][digitized==ii]) == 0:
            temp_list = list(mass_dist.T[kk]\
                                             [digitized==ii])
            temp_list.append(np.zeros(len(bin_nums)))
            mass_dist.T[kk][digitized==ii] = np.array(temp_list)

    # print bin_nums
    # print len(bin_nums)
    
    medians  = np.array([np.median(mass_dist.T[kk][digitized==ii]) \
                for ii in bin_nums])

    # print len(medians)

    if bootstrap == True:
        dist_in_bin    = np.array([(mass_dist.T[kk][digitized==ii]) \
                for ii in bin_nums])
        for vv in range(len(dist_in_bin)):
            if len(dist_in_bin[vv]) == 0:
                dist_in_bin_list = list(dist_in_bin[vv])
                dist_in_bin[vv] = np.zeros(len(dist_in_bin[0]))
        low_err_test   = np.array([np.percentile(astropy.stats.bootstrap\
                        (dist_in_bin[vv],bootnum=1000,bootfunc=np.median),16) \
                        for vv in range(len(dist_in_bin))])
        high_err_test  = np.array([np.percentile(astropy.stats.bootstrap\
                        (dist_in_bin[vv],bootnum=1000,bootfunc=np.median),84) \
                        for vv in range(len(dist_in_bin))])

        med_list    = [[] for yy in range(len(frac_vals))]
        med_list[0] = medians
        med_list[1] = low_err_test
        med_list[2] = high_err_test
        medians     = np.array(med_list)

    return medians    
    

In [41]:
def hist_calcs(mass,bins,dlogM):
    """
    Returns dictionaries with the counts for the upper
        and lower density portions; calculates the 
        three different percentile cuts for each mass
        array given
    
    Parameters
    ----------
    mass: array-like
        A 1D array with log stellar mass values, assumed
        to be an order which corresponds to the ascending 
        densities; (necessary, as the index cuts are based 
        on this)
    bins: array-like
        A 1D array with the values which will be used as the bin edges   
    dlogM: float-like
        The log difference between bin edges
        
    Returns
    -------
    hist_dict_low: dictionary-like
        A dictionary with three keys (the frac vals), with arrays
        as values. The values for the lower density cut
    hist_dict_high: dictionary like
        A dictionary with three keys (the frac vals), with arrays
        as values. The values for the higher density cut
    """
    hist_dict_low  = {}
    hist_dict_high = {}
    frac_val  = np.array([2,4,10])
    frac_dict = {2:0,4:1,10:2}
    
    low_err   = [[] for xx in xrange(len(frac_val))]
    high_err  = [[] for xx in xrange(len(frac_val))]
    
    for ii in frac_val:
        hist_dict_low[ii]  = {}
        hist_dict_high[ii] = {}
    
        frac_data     = int(len(mass)/ii)
        
        frac_mass     = mass[0:frac_data]
        counts, edges = np.histogram(frac_mass,bins)
        low_counts    = (counts/float(len(frac_mass))/dlogM)
        
        low_err[frac_dict[ii]]    = np.sqrt(counts)/len(frac_mass)/dlogM
        
        hist_dict_low[ii]  = low_counts

        frac_mass_2        = mass[-frac_data:]
        counts_2, edges_2  = np.histogram(frac_mass_2,bins)
        high_counts        = (counts_2/float(len(frac_mass_2))/dlogM)

        high_err[frac_dict[ii]] = np.sqrt(counts_2)/len(frac_mass_2)/\
                                            dlogM
        
        hist_dict_high[ii] = high_counts
        
    hist_dict_low['low_err']      = low_err
    hist_dict_high['high_err']    = high_err
    
    return hist_dict_low, hist_dict_high

In [42]:
def mean_bin_mass(mass_dist,bins):
    """
    Returns median distance to Nth nearest neighbor

    Parameters
    ----------
    mass_dist: array-like
        An array with mass values in at index 0 (when transformed) 
    bins: array=like
        A 1D array with the values which will be used as the bin edges     

    Returns
    -------

    """
    edges        = bins

    digitized    = np.digitize(mass_dist.T[0],edges)
    digitized   -= int(1)

    bin_nums          = np.unique(digitized)
    bin_nums_list     = list(bin_nums)

    for jj in range(len(bins)-1):
        if jj not in bin_nums:
            bin_nums_list.append(jj)

    if (len(bins)-1) in bin_nums_list:
        bin_nums_list.remove(len(bins)-1)

    if (len(bins)) in bin_nums_list:
        bin_nums_list.remove(len(bins))

    bin_nums = np.array(bin_nums_list)

    mean_mass = np.array([np.mean(mass_dist.T[0][digitized==ii]) \
                for ii in bin_nums])

    return mean_mass 

In [43]:
def schechter_real_func(mean_of_mass_bin,bins,phi_star,alpha,Mstar):
    M_over_mstar = mean_of_mass_bin/Mstar
    temp_val     = phi_star/Mstar * M_over_mstar**(alpha+1) * \
                        np.exp(- M_over_mstar)
    int_val = [[] for ii in range(len(temp_val))]
    for ii in range(len(temp_val)):
        int_val[ii]      = integrate.quad(temp_val[ii],bins[ii],bins[ii+1])

    return int_val

In [44]:
def find_params(bin_int,mean_mass,count_err):
    """
    Parameters
    ----------

    Returns
    -------
    opt_v: array-like
        Array with three values: phi_star, alpha, and M_star
    res_arr: array-like
        Array with two values: alpha and log_M_star


    """
    xdata = 10**mean_mass
    p0    = (1,-1.05,10**10.64)
    opt_v,est_cov = optimize.curve_fit(schechter_real_func,xdata,\
                            bin_int,p0=p0,sigma=count_err)
    alpha   = opt_v[1]
    log_m_star    = np.log10(opt_v[2])
    res_arr = np.array([alpha,log_m_star])

    return opt_v, res_arr

In [87]:
dirpath  = r"C:\Users\Hannah\Desktop\Vanderbilt_REU\Stellar_mass_env_density"
dirpath += r"\Catalogs\Resolve_plk_5001_so_mvir_scatter_ECO_Mocks_"
dirpath += r"scatter_mocks\Resolve_plk_5001_so_mvir_scatter0p3_ECO_Mocks"

usecols  = (0,1,4,8,13)
dlogM    = 0.2

In [281]:
ECO_cats = (Index(dirpath,'.dat'))

names    = ['ra','dec','Halo_ID','cz','logMstar']

PD = [[] for ii in range(len(ECO_cats))]

for ii in range(len(ECO_cats)):
    temp_PD = (pd.read_csv(ECO_cats[ii],sep="\s+", usecols= usecols,header=None,\
                   skiprows=2,names=names))
    col_list = list(temp_PD)
    col_list[2], col_list[3], col_list[4] = col_list[3], col_list[4], col_list[2]
    temp_PD.ix[:,col_list]
    PD[ii] = temp_PD

PD_comp_1  = [(PD[ii][PD[ii].logMstar >= 9.1]) for ii in range(len(ECO_cats))]
PD_comp  = [(PD_comp_1[ii][PD_comp_1[ii].logMstar <=11.77]) for ii in range(len(ECO_cats))]

[(PD_comp[ii].reset_index(drop=True,inplace=True)) for ii in range(len(ECO_cats))]

[None, None, None, None, None, None, None, None]

In [316]:
# for ii in range(len(ECO_cats)):
#     print len(PD[ii][PD[ii].logMstar>11.77])
# for ii in range(len(ECO_cats)):
#     print PD_comp[ii]


              ra        dec  Halo_ID           cz   logMstar
0     130.953540  49.006256    98893  3475.364999  10.325583
1     157.895819  47.985767    99838  3334.790208   9.543174
2     163.563095  46.988238   100102  3400.305566  10.170545
3     163.603108  47.017633   100102  3400.976649   9.420000
4     131.991198  45.456810   100511  3624.022246  10.166005
5     131.996685  45.408643   100511  3623.632677   9.270000
6     148.491324  40.028513   100619  3474.444815   9.655148
7     157.275878  38.956278   100845  3715.383396  10.158337
8     202.255791   6.056371   115496  4770.049001   9.100746
9     202.334211   5.289930   115502  4810.643649   9.122643
10    202.371412   5.963233   115509  4728.767955  10.450000
11    202.308255   5.762713   115509  4756.531307   9.500000
12    202.441487   5.445382   115509  4742.058636  11.042534
13    202.765759   5.309359   115509  4750.713123   9.500000
14    201.988925   5.988682   115520  4728.720024  10.006373
15    202.278110   5.893

In [283]:
min_max_mass_arr = []

for ii in range(len(PD_comp)):
    min_max_mass_arr.append(max(PD_comp[ii].logMstar))
    min_max_mass_arr.append(min(PD_comp[ii].logMstar))

min_max_mass_arr = np.array(min_max_mass_arr)

bins = Bins_array_create(min_max_mass_arr,dlogM)
bins+= 0.1
bins_list = list(bins)
for ii in bins:
    if ii > 11.7:
        bins_list.remove(ii)

bins = np.array(bins_list)

num_of_bins = int(len(bins) - 1) 

ra_arr  = np.array([(PD_comp[ii].ra) \
    for ii in range(len(PD_comp))])

dec_arr  = np.array([(PD_comp[ii].dec) \
    for ii in range(len(PD_comp))])

cz_arr  = np.array([(PD_comp[ii].cz) \
    for ii in range(len(PD_comp))])

mass_arr  = np.array([(PD_comp[ii].logMstar) \
    for ii in range(len(PD_comp))])

halo_id_arr  = np.array([(PD_comp[ii].Halo_ID) \
    for ii in range(len(PD_comp))])


coords_test = np.array([sph_to_cart(ra_arr[vv],dec_arr[vv],cz_arr[vv]) \
                for vv in range(len(ECO_cats))])

neigh_vals  = np.array([1,2,3,5,10,20])

nn_arr_temp = [[] for uu in xrange(len(coords_test))]
nn_arr      = [[] for xx in xrange(len(coords_test))]
nn_arr_nn   = [[] for yy in xrange(len(neigh_vals))]
nn_idx      = [[] for zz in xrange(len(coords_test))]

for vv in range(len(coords_test)):
    nn_arr_temp[vv] = spatial.cKDTree(coords_test[vv])
    nn_arr[vv] = np.array(nn_arr_temp[vv].query(coords_test[vv],21)[0])
    nn_idx[vv] = np.array(nn_arr_temp[vv].query(coords_test[vv],21)[1])
    

nn_specs       = [(np.array(nn_arr).T[ii].T[neigh_vals].T) for ii in \
                    range(len(coords_test))]
nn_mass_dist   = np.array([(np.column_stack((mass_arr[qq],nn_specs[qq]))) \
                    for qq in range(len(coords_test))])

nn_neigh_idx      = np.array([(np.array(nn_idx).T[ii].T[neigh_vals].T) for ii in \
                    range(len(coords_test))])

In [315]:
test = []

for ii in range(len(halo_id_arr[0])):
    a = np.unique(halo_id_arr[0][nn_neigh_idx[0].T[0]][halo_id_arr[0][nn_neigh_idx[0].T[0]].keys()[ii]])
    b = halo_id_arr[0][halo_id_arr[0].keys()][ii]
    test.append(a==b)
    
    
#####THIS WON'T WORK, BUT THE ABOVE DOES; SAYS LIST INDEX OUT OF RANGE    
# for ii in range(4):
#     test[ii] = [np.unique(halo_id_arr[0][nn_neigh_idx[0].T[0]][halo_id_arr[0][nn_neigh_idx[0].T[0]].keys()[ii]])\
#                == halo_id_arr[0][halo_id_arr[0].keys()][ii]]
    
print test    
    
                
# for ii in range(4):
#     print 'first_val'
#     a = np.unique(halo_id_arr[0][nn_neigh_idx[0].T[0]][halo_id_arr[0][nn_neigh_idx[0].T[0]].keys()[ii]])
#     print 'sec_val'
#     b = halo_id_arr[0][halo_id_arr[0].keys()][ii]
#     print a == b

    
# print halo_id_arr[0][nn_neigh_idx[0].T[0]].keys()

# print halo_id_arr[0].keys()
# [halo_id_arr[0].keys()]
                 
# print halo_id_arr[0][nn_idx[0].T[0]][ii]
# print halo_id_arr[0][halo_id_arr[0].keys()[ii]]

# print test    

neigh_dict = {1:0,2:1,3:2,5:3,10:4,20:5}

##nn_neigh_idx[aa] = mock

##nn_neigh_idx[aa][bb] = row with 6 nearest neighbor indices

## nn_neigh_idx[aa].T[neigh_dict[x]] = array with nearest neighbor indices for every galaxies x nearest neighbor
    #x = 1,2,3,5,10,20   

IndexError: list assignment index out of range

In [52]:
nn_dist    = {}
nn_dens    = {}
mass_dat   = {}
ratio_info = {}

mass_freq  = [[] for xx in xrange(len(coords_test))]

for ii in range(len(coords_test)):
    nn_dist[ii]    = {}
    nn_dens[ii]    = {}
    mass_dat[ii]   = {}
    ratio_info[ii] = {}

    nn_dist[ii]['mass'] = nn_mass_dist[ii].T[0]

    for jj in range(len(neigh_vals)):
        nn_dist[ii][(neigh_vals[jj])]  = np.array(nn_mass_dist[ii].T\
                                            [range(1,len(neigh_vals)+1)[jj]])
        nn_dens[ii][(neigh_vals[jj])]  = np.column_stack((nn_mass_dist[ii].T\
                                            [0],calc_dens(neigh_vals[jj],\
                                            nn_mass_dist[ii].T[range(1,len\
                                                (neigh_vals)+1)[jj]])))

        idx = np.array([nn_dens[ii][neigh_vals[jj]].T[1].argsort()])
        mass_dat[ii][(neigh_vals[jj])] = (nn_dens[ii][neigh_vals[jj]]\
                                            [idx].T[0])

        bin_centers, mass_freq[ii], ratio_info[ii][neigh_vals[jj]] = \
                            plot_calcs(mass_dat[ii][neigh_vals[jj]],bins,dlogM)

all_mock_meds = [[] for xx in range(len(nn_mass_dist))]
all_mock_mass_means = [[] for xx in range(len(nn_mass_dist))]

for vv in range(len(nn_mass_dist)):
    all_mock_meds[vv] = np.array([bin_func(nn_mass_dist[vv],bins,(jj+1)) \
                                for jj in range(len(nn_mass_dist[vv].T)-1)])
    all_mock_mass_means[vv] = (mean_bin_mass(nn_mass_dist[vv],bins))  
    
med_plot_arr = [([[] for yy in xrange(len(nn_mass_dist))]) \
                                            for xx in xrange(len(neigh_vals))]

for ii in range(len(neigh_vals)):
    for jj in range(len(nn_mass_dist)):
        med_plot_arr[ii][jj] = all_mock_meds[jj][ii]    

# for ii in range(len(neigh_vals)):
#     for jj in range(len(nn_mass_dist)):
#         print len(all_mock_meds[jj][ii])

# mass_freq_plot  = (np.array(mass_freq))
# max_lim = [[] for xx in range(len(mass_freq_plot.T))]
# min_lim = [[] for xx in range(len(mass_freq_plot.T))]
# for jj in range(len(mass_freq_plot.T)):
#     max_lim[jj] = max(mass_freq_plot.T[jj])
#     min_lim[jj] = min(mass_freq_plot.T[jj])

In [53]:
mass_freq



[array([[ 0.86946387,  0.72649573,  0.63636364,  0.56876457,  0.47707848,
          0.48018648,  0.44289044,  0.35042735,  0.24941725,  0.11810412,
          0.06216006,  0.01631702,  0.001554  ],
        [ 0.02599181,  0.02375895,  0.02223635,  0.02102214,  0.01925332,
          0.01931593,  0.01855064,  0.01650098,  0.01392111,  0.00957951,
          0.00694971,  0.00356066,  0.00109885]]),
 array([[ 0.88682746,  0.72820037,  0.6270872 ,  0.52504638,  0.5       ,
          0.48701299,  0.42764378,  0.35992579,  0.25974026,  0.1270872 ,
          0.05473098,  0.01113173,  0.00463822],
        [ 0.02868205,  0.02599059,  0.02411874,  0.02206935,  0.02153652,
          0.02125499,  0.01991736,  0.01827246,  0.01552245,  0.01085779,
          0.00712537,  0.00321345,  0.00207427]]),
 array([[  8.70452227e-01,   7.30193812e-01,   6.22237334e-01,
           5.37232234e-01,   5.06630398e-01,   4.82828970e-01,
           4.41176471e-01,   3.75722543e-01,   2.22713363e-01,
           1.283577

In [54]:
eco_path  = r"C:\Users\Hannah\Desktop\Vanderbilt_REU\Stellar_mass_env_density"
eco_path += r"\Catalogs\ECO_true"
eco_cols  = np.array([0,1,2,4])

In [55]:
ECO_true = (Index(eco_path,'.txt'))
names    = ['ra','dec','cz','logMstar']
PD_eco   = pd.read_csv(ECO_true[0],sep="\s+", usecols=(eco_cols),header=None,\
                skiprows=1,names=names)
eco_comp = PD_eco[PD_eco.logMstar >= 9.1]

ra_eco   = (np.array(eco_comp)).T[0]
dec_eco  = (np.array(eco_comp)).T[1] 
cz_eco   = (np.array(eco_comp)).T[2] 
mass_eco = (np.array(eco_comp)).T[3]

coords_eco        = sph_to_cart(ra_eco,dec_eco,cz_eco)
eco_neighbor_tree = spatial.cKDTree(coords_eco)
eco_tree_dist     = np.array(eco_neighbor_tree.query(coords_eco,\
                    (neigh_vals[-1]+1))[0])

eco_mass_dist = np.column_stack((mass_eco,eco_tree_dist.T[neigh_vals].T))
##range 1,7 because of the six nearest neighbors (and fact that 0 is mass)
##the jj is there to specify which index in the [1,6] array
eco_dens = ([calc_dens(neigh_vals[jj],\
            (eco_mass_dist.T[range(1,7)[jj]])) for jj in range\
            (len(neigh_vals))])

eco_mass_dens = [(np.column_stack((mass_eco,eco_dens[ii]))) for ii in \
                range(len(neigh_vals))]
eco_idx  = [(eco_mass_dens[jj].T[1].argsort()) for jj in \
            range(len(neigh_vals))]
eco_mass_dat  = [(eco_mass_dens[jj][eco_idx[jj]].T[0]) for jj in \
                range(len(neigh_vals))]

eco_ratio_info    = [[] for xx in xrange(len(eco_mass_dat))]


for qq in range(len(eco_mass_dat)):
    bin_centers, eco_freq, eco_ratio_info[qq] = plot_calcs(eco_mass_dat[qq],\
                                    bins,dlogM)

eco_medians   = [[] for xx in xrange(len(eco_mass_dat))]    

for jj in (range(len(eco_mass_dat))):
    eco_medians[jj] = np.array(bin_func(eco_mass_dist,bins,(jj+1),\
        bootstrap=True))

In [56]:
hist_low_info  = {}
hist_high_info = {}

for ii in xrange(len(coords_test)):
    hist_low_info[ii]  = {}
    hist_high_info[ii] = {}
    
    for jj in range(len(neigh_vals)):
        hist_low_info[ii][neigh_vals[jj]],hist_high_info[ii][neigh_vals[jj]] \
        = hist_calcs(mass_dat[ii][neigh_vals[jj]],bins,dlogM)
        
frac_vals     = [2,4,10]
hist_low_arr  = [[[] for yy in xrange(len(nn_mass_dist))] for xx in \
    xrange(len(neigh_vals))]
hist_high_arr = [[[] for yy in xrange(len(nn_mass_dist))] for xx in \
    xrange(len(neigh_vals))]

for ii in range(len(neigh_vals)):
    for jj in range(len(nn_mass_dist)):
        hist_low_arr[ii][jj]  = (hist_low_info[jj][neigh_vals[ii]])
        hist_high_arr[ii][jj] = (hist_high_info[jj][neigh_vals[ii]])
        
        
        plot_low_hist  = [[[[] for yy in xrange(len(nn_mass_dist))] \
                         for zz in xrange(len(frac_vals))] for xx in \
                         xrange(len(hist_low_arr))]
        
        plot_high_hist = [[[[] for yy in xrange(len(nn_mass_dist))] \
                 for zz in xrange(len(frac_vals))] for xx in \
                 xrange(len(hist_high_arr))]

for jj in range(len(nn_mass_dist)):
    for hh in range(len(frac_vals)):
        for ii in range(len(neigh_vals)):
            plot_low_hist[ii][hh][jj]  = hist_low_arr[ii][jj][frac_vals[hh]]        
            plot_high_hist[ii][hh][jj] = hist_high_arr[ii][jj][frac_vals[hh]] 

In [58]:
eco_low  = {}
eco_high = {}
for jj in range(len(neigh_vals)):
    eco_low[neigh_vals[jj]]  = {}
    eco_high[neigh_vals[jj]] = {}
    eco_low[neigh_vals[jj]], eco_high[neigh_vals[jj]] = hist_calcs\
    (eco_mass_dat[jj],bins,dlogM)

In [62]:
hist_low_info



{0: {1: {2: array([ 0.89679826,  0.78023003,  0.68697544,  0.58594964,  0.49891203,
           0.49735779,  0.46316444,  0.3139571 ,  0.1911719 ,  0.06372397,
           0.0217594 ,  0.        ,  0.        ]),
   4: array([ 1.02922886,  0.8550995 ,  0.6778607 ,  0.60634328,  0.50373134,
           0.47574627,  0.42910448,  0.24564677,  0.12748756,  0.04353234,
           0.00621891,  0.        ,  0.        ]),
   10: array([ 1.11975117,  0.97200622,  0.68429238,  0.53654743,  0.52099533,
           0.41990669,  0.46656299,  0.17884914,  0.0777605 ,  0.00777605,
           0.0155521 ,  0.        ,  0.        ]),
   'low_err': [array([ 0.0373342 ,  0.03482337,  0.03267609,  0.03017794,  0.02784655,
            0.02780315,  0.0268304 ,  0.02208994,  0.01723739,  0.00995201,
            0.00581545,  0.        ,  0.        ]),
    array([ 0.05657153,  0.05156444,  0.04591052,  0.04342114,  0.03957687,
            0.03846181,  0.0365278 ,  0.02763742,  0.01991021,  0.01163451,
            0.